# gemini

## google api docs
- [generativelanguage](https://ai.google.dev/api/python/google/ai/generativelanguage/)

### 安装等
```sh
source ~/llm-env/llm-jupiter-2/bin/activate
pip install google-generativeai
pip install -q -U google-generativeai
```


## 基本
### models
- gemini-pro
- gemini-1.5-flash


### methods
- model.generate_content()
- model.start_chat()

### run
- 使用python venv: python3.11.6 


## 问题
### 安全设置和candidates
- 由于使用的是免费api，文档里面提到即使安全设置为BLOCK_NONE，也会触发安全信息。
- 这个时候返回内容在 candidates 里面。而response.parts为空
- [参考文档](https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/configure-safety-attributes?hl=zh-cn)


In [1]:
import env
import utils
# Generation config
config = {"max_output_tokens": 2048, "temperature": 0.4, "top_p": 1, "top_k": 32}

In [11]:
# gemini-pro
import google.generativeai as genai
import os

genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))
# model = genai.GenerativeModel(model_name = "gemini-pro")
model = genai.GenerativeModel("gemini-1.5-flash")

In [24]:
# generate_content 方法
# 可以一次传入多个prompts
prompt_parts = [
    "Write a song in Chinese in 100 words",
    "Write a Python function, and explain it to me"
]

response = model.generate_content(prompt_parts)

In [26]:
print(response.text)

## Chinese Song (100 words)

(Verse 1)
月光洒在窗台，
夜风轻轻吹过。
思绪如云飘渺，
思念化作星河。

(Chorus)
远方的你，
是否也抬头望月？
是否也心怀念，
这份爱，永不落。

(Verse 2)
花开花落，
季节轮回不停。
我的心，却始终，
为你而跳动。

(Chorus)
远方的你，
是否也抬头望月？
是否也心怀念，
这份爱，永不落。

## Python Function

```python
def greet(name):
  """
  This function greets a person by name.

  Args:
    name: The name of the person to greet.

  Returns:
    A string containing the greeting message.
  """
  return f"Hello, {name}!"

# Example usage
greeting = greet("Alice")
print(greeting) # Output: Hello, Alice!
```

**Explanation:**

* **`def greet(name):`**: This line defines a function named `greet` that takes one argument called `name`.
* **`""" ... """`**: This is a docstring, which describes what the function does.
* **`return f"Hello, {name}!"`**: This line returns a string containing the greeting message, using an f-string to insert the `name` value.
* **`# Example usage`**: This line is a comment that explains how to use the function.
* **`greeting = greet("Alice")`**: This lin

In [27]:
# 使用chat模型 - 
chat = model.start_chat(history=[])

response = chat.send_message("Hello, my name is Bell.")
utils.to_markdown(response.text)

> Hello Bell! 👋 It's nice to meet you. 😊 What can I do for you today? 


In [28]:
response = chat.send_message("Yes. Do you know how to make money?")
utils.to_markdown(response.text)

> As a large language model, I don't have the ability to make money in the way humans do. I don't have a job, a bank account, or the ability to buy or sell things.  
> 
> However, I can help you with information that might be useful if you're looking to make money.  What are your thoughts about earning money? Are you looking for a specific job, or do you have an idea for a business? 
> 
> Tell me more about what you're interested in, and I'll do my best to help!  


In [29]:
response = chat.send_message("Do you know who am I?")
utils.to_markdown(response.text)

> As an AI, I don't have a memory of past conversations or personal information about individuals.  So, I don't know who you are, Bell. 
> 
> If you'd like to tell me more about yourself, I'd be happy to learn!  What are some things about you that you'd like to share? 😊 


In [30]:
# check the history
for message in chat.history:
  display(utils.to_markdown(f'**{message.role}**: {message.parts[0].text}'))

> **user**: Hello, my name is Bell.

> **model**: Hello Bell! 👋 It's nice to meet you. 😊 What can I do for you today? 


> **user**: Yes. Do you know how to make money?

> **model**: As a large language model, I don't have the ability to make money in the way humans do. I don't have a job, a bank account, or the ability to buy or sell things.  
> 
> However, I can help you with information that might be useful if you're looking to make money.  What are your thoughts about earning money? Are you looking for a specific job, or do you have an idea for a business? 
> 
> Tell me more about what you're interested in, and I'll do my best to help!  


> **user**: Do you know who am I?

> **model**: As an AI, I don't have a memory of past conversations or personal information about individuals.  So, I don't know who you are, Bell. 
> 
> If you'd like to tell me more about yourself, I'd be happy to learn!  What are some things about you that you'd like to share? 😊 


In [31]:
# chat - with history/context
chat = model.start_chat(history=[
    {
        'role': 'user',
        'parts': ['You are a helpful assistant.']
    },
    {
        'role': 'model',
        'parts': ['Sure.'],
    },
])


resp = chat.send_message(
    content='hi, how are you?',
)

print(resp.text)

I'm doing well, thank you for asking! How are you today? 



In [10]:
context = [ {'role':'user', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages